In [ ]:
import sqlite3

# 1. DBへのコネクションを確立
DB_NAME = 'google_repos_data.db' # 確定したDBファイル名
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# 2. テーブルの定義とSQLの実行
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    language TEXT,
    stars INTEGER
)
"""
cursor.execute(CREATE_TABLE_SQL)

# 3. コミット（テーブル作成を確定）
conn.commit()

print(f"データベース '{DB_NAME}' に接続し、テーブル 'repos' を作成しました。")

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re # スター数を抽出するために正規表現を使用

# 1. スクレイピング対象のURL
URL = "https://github.com/google?tab=repositories" # 確定したURL
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 2. HTMLの取得と解析
response = requests.get(URL, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

# リポジトリ一覧のコンテナ要素を特定
repo_list = soup.find_all('li', {'class': 'Box-row'})

extracted_data = []

# 3. データの抽出（最初の5件ほどをテスト抽出）
for i, repo in enumerate(repo_list):
    if i >= 5: 
        break
        
    # リポジトリ名の抽出
    name_tag = repo.find('a', {'itemprop': 'name codeRepository'})
    repo_name = name_tag.text.strip() if name_tag else 'N/A'
    
    # 主要言語の抽出
    language_tag = repo.find('span', {'itemprop': 'programmingLanguage'})
    main_language = language_tag.text.strip() if language_tag else 'N/A'
    
    # スター数の抽出 (href属性が /stargazers で終わるaタグを探す)
    star_tag = repo.find('a', {'href': re.compile(r'/stargazers$')})
    stars = 0
    if star_tag:
        stars_text = star_tag.text.strip().replace(',', '')
        if stars_text:
             stars = int(stars_text)
    
    extracted_data.append({
        'name': repo_name,
        'language': main_language,
        'stars': stars
    })

print(f"【抽出データ確認】最初の {len(extracted_data)} 件:")
print(extracted_data)

# 全リポジトリのリストを次のセルに引き渡すためにグローバル変数として保持


In [ ]:
INSERT_SQL = """
INSERT INTO repos (name, language, stars) VALUES (?, ?, ?)
"""

print(f"全 {len(repo_list)} 件のリポジトリ情報を取得し、DBに挿入します。")

# 既存データをクリア
cursor.execute("DELETE FROM repos") 
conn.commit()

# 全リポジトリに対してループを実行し、挿入
for i, repo in enumerate(repo_list):
    # サーバー負荷軽減
    time.sleep(1) 
    
    # --- データ抽出ロジック（再掲） ---
    name_tag = repo.find('a', {'itemprop': 'name codeRepository'})
    repo_name = name_tag.text.strip() if name_tag else 'N/A'
    
    language_tag = repo.find('span', {'itemprop': 'programmingLanguage'})
    main_language = language_tag.text.strip() if language_tag else 'N/A'
    
    star_tag = repo.find('a', {'href': re.compile(r'/stargazers$')})
    stars = 0
    if star_tag:
        stars_text = star_tag.text.strip().replace(',', '')
        if stars_text:
             stars = int(stars_text)

    data_to_insert = (repo_name, main_language, stars)
    
    try:
        cursor.execute(INSERT_SQL, data_to_insert)
        if (i + 1) % 10 == 0:
            print(f"{i + 1} 件挿入: {repo_name}")

    except Exception as e:
        print(f"エラー: {e}")

# データの永続化
conn.commit()
print("すべてのデータをDBにコミットしました。")

In [ ]:
#保存したデータをSELECT文で表示する
SELECT_ALL_SQL = "SELECT * FROM repos ORDER BY stars DESC LIMIT 10" # スター数が多い順に10件表示
cursor.execute(SELECT_ALL_SQL)

# 結果の取得
results = cursor.fetchall()

print("\n【DBから取得したデータ (スター数順 上位10件)】")
print("-----------------------------------------------------------------")
print(f"{'ID':<4} {'Name':<35} {'Language':<15} {'Stars':<8}")
print("-----------------------------------------------------------------")
for row in results:
    repo_id, name, language, stars = row
    print(f"{repo_id:<4} {name:<35} {language:<15} {stars:<8,}")
print("-----------------------------------------------------------------")


conn.close()